# PicoTuri-EditJudge Baseline

End-to-end baseline: load Pico-Banana manifests, compute Turi Create image similarity, fuse with instruction TF-IDF, and train a logistic regression model.

> **Environment**: macOS, Python 3.8 (x86_64) under Rosetta if you are on Apple Silicon.
>
> **License reminder**: Pico-Banana-400K is CC BY-NC-ND 4.0. Do not redistribute media.

In [ ]:
import pandas as pd

from pathlib import Path

from src.features_image import compute_pair_similarity
from src.train.baseline import train_baseline_model

PAIRS_PATH = Path("data/manifests/sample_pairs.csv")
USE_TURI = False  # flip to True when turicreate is installed
SEED = 13

In [ ]:
pairs = pd.read_csv(PAIRS_PATH)
pairs.head()

In [ ]:
if USE_TURI:
    image_pairs = list(zip(pairs["original_image"], pairs["edited_image"]))
    similarities = compute_pair_similarity(image_pairs)
    pairs["image_similarity"] = similarities
    pairs.to_csv(PAIRS_PATH.with_name("pairs_with_similarity.csv"), index=False)
    print("Computed Turi Create similarities and wrote updated manifest.")
else:
    print("Skipping Turi Create similarity computation; using precomputed values.")

In [ ]:
artifacts, (X_test, y_test) = train_baseline_model(PAIRS_PATH, test_size=0.3, seed=SEED)
metrics = artifacts.metrics
metrics

In [ ]:
display(pd.DataFrame(X_test))
display(y_test.reset_index(drop=True))

## Next steps

- Replace the precomputed similarity column with live Turi Create features.
- Export the image encoder to Core ML and integrate with the SwiftUI demo.
- Track calibration experiments (Platt scaling, isotonic regression) for Milestone 0.5.